# Docs 1개~4개
- 정답(1)과 랜덤개수의 오답문서(0~3)을 섞어 doc를 생성한다.
- docs는 1 ~ 4개 사이를 가지게 된다.

In [ ]:
import random
import ast
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import os
import re
from tqdm import tqdm

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# klue_mrc_context_question_negative_samples.csv
!gdown 1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc

Downloading...
From (original): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc
From (redirected): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc&confirm=t&uuid=f17721dd-59ec-4ada-a1df-3cc6cd32028c
To: /content/klue_mrc_context_question_negative_samples.csv
100% 161M/161M [00:00<00:00, 162MB/s]


In [ ]:
# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv('klue_mrc_context_question_negative_samples.csv')

# 1000~1299번째 행 추출
df_300 = df.iloc[1000:1300].reset_index(drop=True)
df_300

,context,question,negative_samples
0,경기도가 주최하고 경기콘텐츠진흥원이 주관하는 ‘경기도 1인 크리에이터 아카데미’가 ...,경기도 1인 크리에이터 아카데미’의 수강료는?,['경기도와 경기콘텐츠진흥원(원장 송경희)이 ‘경기 메이커 프로젝트 지원’ 사업을 ...
1,한국은 경제협력개발기구(OECD) 회원국 가운데 영아(0~2세)의 어린이집 이용률(...,보육료가 차등지급되는 국가는?,['부분적 선별복지로의 전환을 알리는 신호탄이 무상보육체계 개편이다. 정부는 그동안...
2,웹젠(대표 김태영)이 PC 온라인 게임 'R2'의 ‘오리지널 리부트 서버’를 초기화...,정기정검이 끝나는 날짜는?,['르노삼성자동차(대표이사 도미닉시뇨라)는 민족대명절 추석을 맞아 오는 23일부터 ...
3,"충청남도 천안군 수신면에서 태어났다. 가난한 소작농 집안이었지만, 어릴 때부터 총명...",천안군당의 사상은?,['충남 예산군 출신으로 일본에 유학하여 주오 대학 경제과를 나왔다. 이후 귀국하여...
4,환율 변동으로 피해를 입은 중소기업에 긴급경영안정자금이 지원된다. 또 중소기업의 선...,원-달러 환율이 1100원 아래로 내려간 시기는?,['2012년부터 시작된 원·달러 환율의 추세적 하락이 과거 원화 강세기보다 한국 ...
...,...,...,...
295,서울시교육청은 교육 소외계층에 교육 기회를 제공하기 위해 방송통신중학교를 내년 3월...,방송통신중학교에 내년에 재학할 것으로 예측되는 학생 수는?,['한국방송통신대가 개교 41년 만에 처음으로 올해 2학기 신입생 모집에 나섰다. ...
296,성인인 아들딸 이름으로 5000만원 이하의 본인 돈을 예금해도 세금에 변화가 없다면...,자녀가 어른이 아닌 경우 증여세 감면 범위는 얼마인가?,['자녀가 어릴 때부터 계획을 세워 단계적으로 재산을 물려주는 ‘계획증여’가 늘고 ...
297,저출산에 따른 인구 정체로 어려움을 겪고 있는 생활용품·어린이용품 업체들이 ‘베이비...,이른둥이 전용 기저귀를 국내에 처음 선보인 기업은?,"[""최근 오랜 불경기로 인해 영유아기 자녀를 둔 부모들 사이에서 ‘가성비’와 ‘실용..."
298,카리루는 파이썬으로 구축된 클라우드 서버인 구글 앱 엔진과 일반 서버의 하이브리드 ...,2010년에 카리루를 차단한 도서관은?,['서울 신촌오거리에서 17년간 터줏대감으로 자리를 지킨 그랜드마트 신촌점은 지난해...


In [ ]:
# 각 row에 대해 docs 리스트를 만들고 DataFrame에 저장
def make_docs_variable(row):
    # negative_samples가 str이면 리스트로 변환
    negatives = row['negative_samples']
    if isinstance(negatives, str):
        negatives = ast.literal_eval(negatives)
    # 오답 개수(0~3개) 랜덤 선택
    n_neg = random.randint(0, min(3, len(negatives)))
    sampled_neg = random.sample(negatives, n_neg) if n_neg > 0 else []
    docs = [row['context']] + sampled_neg
    random.shuffle(docs)
    return docs

df_300['docs'] = df_300.apply(make_docs_variable, axis=1)
df_300[['question', 'docs']].head()

,question,docs
0,경기도 1인 크리에이터 아카데미’의 수강료는?,[경기도와 경기콘텐츠진흥원(원장 송경희)이 ‘경기 메이커 프로젝트 지원’ 사업을 통...
1,보육료가 차등지급되는 국가는?,[부분적 선별복지로의 전환을 알리는 신호탄이 무상보육체계 개편이다. 정부는 그동안 ...
2,정기정검이 끝나는 날짜는?,[웹젠(대표 김태영)이 PC 온라인 게임 'R2'의 ‘오리지널 리부트 서버’를 초기...
3,천안군당의 사상은?,[1945년 8월 15일 일제로부터 해방되자 가장 먼저 활동을 시작한 세력은 여운형...
4,원-달러 환율이 1100원 아래로 내려간 시기는?,[원·달러 환율이 1100원 선에 바짝 다가섰다. 지난 6일 주형환 기획재정부 1차...


In [ ]:
df_300['docs'].apply(len)

,docs
0,4
1,3
2,1
3,2
4,3
...,...
295,2
296,1
297,2
298,3


In [ ]:
df_300['docs'].apply(len).value_counts().sort_index()

,count
docs,
1,68
2,71
3,79
4,82


In [ ]:
system_prompt = """당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요."""

user_prompts = []
system_prompts = []

for idx, row in df_300.iterrows():
    docs = row['docs']  # 이미 섞인 리스트
    # 문서에 번호 붙이기
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(docs)]
    docs_str = '\n'.join(doc_items)
    # user_prompt 생성
    user_prompt = f"""질문: {row['question']}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""
    # 리스트에 추가
    user_prompts.append(user_prompt)
    system_prompts.append(system_prompt)

# 예시 출력
print("system_prompt:")
print(system_prompts[0])
print("\nuser_prompt:")
print(user_prompts[0])

system_prompt:
당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.

user_prompt:
질문: 경기도 1인 크리에이터 아카데미’의 수강료는?

docs:
doc1: 경기도와 경기콘텐츠진흥원(원장 송경희)이 ‘경기 메이커 프로젝트 지원’ 사업을 통해 경기도 메이커를 지원한다. 오는 6월 10일까지 신청자를 모집하고, 서류 및 발표심사를 거쳐 15건 내외의 프로젝트를 선발할 예정이다. ‘경기 메이커 프로젝트 지원’은 경기도와 경기콘텐츠진흥원이 3년째 진행 중인 「경기 메이커스 활성화 지원사업」이다. 해당 사업은 경기도내 창의적이고 도전적인 메이커들의 메이킹 활동을 지원한다. 이번 ‘경기 메이커 프로젝트 지원’ 선발대상에게는 ‘메이커 프로젝트’ 개발에 필요한 지원금을 최대 500만원 이내로 차등 지급할 예정이다. 뿐만 아니라 전문 CEO들의 컨설팅 프로그램이 준비되어있다. 신청자격은 ‘메이커 프로젝트’를 구상중인 메이커라면 공방, 작가, 메이커, 일반인, 스타트업 누구나 제한 없이 지원할 수 있다. 경기콘텐츠진흥원 관계자는 “올해 경기 메이커스 활성화 지원사업은 ‘메이커의 메이킹 활동’을 더욱 집중적으로 지원할 예정”이라며 “적극적인 참여를 통해 경기도를 대표하는 다양한 메이커가 발굴되기를 바란다”고 밝혔다.
do

In [ ]:
df_300['user_prompt'] = user_prompts
df_300['system_prompt'] = system_prompts
df_300[['question', 'docs', 'system_prompt', 'user_prompt']].head()

,question,docs,system_prompt,user_prompt
0,경기도 1인 크리에이터 아카데미’의 수강료는?,[경기도와 경기콘텐츠진흥원(원장 송경희)이 ‘경기 메이커 프로젝트 지원’ 사업을 통...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 경기도 1인 크리에이터 아카데미’의 수강료는?\n\ndocs:\ndoc1: ...
1,보육료가 차등지급되는 국가는?,[부분적 선별복지로의 전환을 알리는 신호탄이 무상보육체계 개편이다. 정부는 그동안 ...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 보육료가 차등지급되는 국가는?\n\ndocs:\ndoc1: 부분적 선별복지로...
2,정기정검이 끝나는 날짜는?,[웹젠(대표 김태영)이 PC 온라인 게임 'R2'의 ‘오리지널 리부트 서버’를 초기...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 정기정검이 끝나는 날짜는?\n\ndocs:\ndoc1: 웹젠(대표 김태영)이...
3,천안군당의 사상은?,[1945년 8월 15일 일제로부터 해방되자 가장 먼저 활동을 시작한 세력은 여운형...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 천안군당의 사상은?\n\ndocs:\ndoc1: 1945년 8월 15일 일제...
4,원-달러 환율이 1100원 아래로 내려간 시기는?,[원·달러 환율이 1100원 선에 바짝 다가섰다. 지난 6일 주형환 기획재정부 1차...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 원-달러 환율이 1100원 아래로 내려간 시기는?\n\ndocs:\ndoc1...


In [ ]:
# 7~8분 소요
# TPM(Token Per Minute) Rate Limit
# - gpt-4o(Tier1) TPM 30000
# - gpt-4.1-mini-2025-04-14(Tier1) TPM 200000
import time

client = OpenAI()

# 응답 저장 리스트
responses = []

for i, user_prompt in enumerate(tqdm(user_prompts, desc="응답 생성 중", ncols=80)):
    response = client.chat.completions.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    answer = response.choices[0].message.content
    responses.append(answer)
    # time.sleep(0.2)

응답 생성 중: 100%|███████████████████████████| 300/300 [11:51<00:00,  2.37s/it]


In [ ]:
def extract_doc_citations(text):
    """
    [[doc1]], [[doc2]], ... 형식의 인용 문서번호만 리스트로 반환
    """
    # 정규표현식: 대소문자 구분 없이 [[doc숫자]]
    citations = re.findall(r'\[\[(doc\d+)\]\]', text, flags=re.IGNORECASE)
    return list(set(citations))  # 중복 제거, 원하면 정렬도 가능

In [ ]:
# 1. answer 컬럼 추가
df_300['answer'] = responses

# 2. doc_citations 컬럼 추가
df_300['doc_citations'] = df_300['answer'].apply(extract_doc_citations)

# 3. 결과 일부 확인
df_300[['question', 'answer', 'docs', 'doc_citations']].head()

,question,answer,docs,doc_citations
0,경기도 1인 크리에이터 아카데미’의 수강료는?,"‘경기도 1인 크리에이터 아카데미’의 교육비는 무료이며, 보증금 10만원은 교육 7...",[경기도와 경기콘텐츠진흥원(원장 송경희)이 ‘경기 메이커 프로젝트 지원’ 사업을 통...,[doc3]
1,보육료가 차등지급되는 국가는?,보육료가 차등지급되는 국가는 한국입니다. 한국 정부는 그동안 대상에 상관없이 동일한...,[부분적 선별복지로의 전환을 알리는 신호탄이 무상보육체계 개편이다. 정부는 그동안 ...,"[doc1, doc3]"
2,정기정검이 끝나는 날짜는?,정기점검이 끝나는 날짜는 9월 10일(목)입니다. 이는 8월 27일(목) 서버 초기...,[웹젠(대표 김태영)이 PC 온라인 게임 'R2'의 ‘오리지널 리부트 서버’를 초기...,[doc1]
3,천안군당의 사상은?,"천안군당의 사상에 대해 직접적으로 명시된 내용은 docs에 없습니다. 다만, doc...",[1945년 8월 15일 일제로부터 해방되자 가장 먼저 활동을 시작한 세력은 여운형...,[doc2]
4,원-달러 환율이 1100원 아래로 내려간 시기는?,원-달러 환율이 1100원 아래로 내려간 시기에 대한 명확한 시점은 docs에 직접...,[원·달러 환율이 1100원 선에 바짝 다가섰다. 지난 6일 주형환 기획재정부 1차...,[doc1]


In [ ]:
# 인용 문서 정보가 없는 행 제거 (즉, doc_citations가 빈 리스트인 행 제거)
df_with_citation = df_300[df_300['doc_citations'].apply(lambda x: len(x) > 0)].reset_index(drop=True)

print(f"인용 정보가 있는 행만 남은 데이터 shape: {df_with_citation.shape}")

인용 정보가 있는 행만 남은 데이터 shape: (297, 8)


In [ ]:
df_with_citation[['question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']]

,question,docs,user_prompt,system_prompt,answer,doc_citations
0,경기도 1인 크리에이터 아카데미’의 수강료는?,[경기도와 경기콘텐츠진흥원(원장 송경희)이 ‘경기 메이커 프로젝트 지원’ 사업을 통...,질문: 경기도 1인 크리에이터 아카데미’의 수강료는?\n\ndocs:\ndoc1: ...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","‘경기도 1인 크리에이터 아카데미’의 교육비는 무료이며, 보증금 10만원은 교육 7...",[doc3]
1,보육료가 차등지급되는 국가는?,[부분적 선별복지로의 전환을 알리는 신호탄이 무상보육체계 개편이다. 정부는 그동안 ...,질문: 보육료가 차등지급되는 국가는?\n\ndocs:\ndoc1: 부분적 선별복지로...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",보육료가 차등지급되는 국가는 한국입니다. 한국 정부는 그동안 대상에 상관없이 동일한...,"[doc1, doc3]"
2,정기정검이 끝나는 날짜는?,[웹젠(대표 김태영)이 PC 온라인 게임 'R2'의 ‘오리지널 리부트 서버’를 초기...,질문: 정기정검이 끝나는 날짜는?\n\ndocs:\ndoc1: 웹젠(대표 김태영)이...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",정기점검이 끝나는 날짜는 9월 10일(목)입니다. 이는 8월 27일(목) 서버 초기...,[doc1]
3,천안군당의 사상은?,[1945년 8월 15일 일제로부터 해방되자 가장 먼저 활동을 시작한 세력은 여운형...,질문: 천안군당의 사상은?\n\ndocs:\ndoc1: 1945년 8월 15일 일제...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","천안군당의 사상에 대해 직접적으로 명시된 내용은 docs에 없습니다. 다만, doc...",[doc2]
4,원-달러 환율이 1100원 아래로 내려간 시기는?,[원·달러 환율이 1100원 선에 바짝 다가섰다. 지난 6일 주형환 기획재정부 1차...,질문: 원-달러 환율이 1100원 아래로 내려간 시기는?\n\ndocs:\ndoc1...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",원-달러 환율이 1100원 아래로 내려간 시기에 대한 명확한 시점은 docs에 직접...,[doc1]
...,...,...,...,...,...,...
292,방송통신중학교에 내년에 재학할 것으로 예측되는 학생 수는?,[서울시교육청은 교육 소외계층에 교육 기회를 제공하기 위해 방송통신중학교를 내년 3...,질문: 방송통신중학교에 내년에 재학할 것으로 예측되는 학생 수는?\n\ndocs:\...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","방송통신중학교에 내년에 재학할 것으로 예측되는 학생 수는 약 2,618명 정도입니다...",[doc1]
293,자녀가 어른이 아닌 경우 증여세 감면 범위는 얼마인가?,[성인인 아들딸 이름으로 5000만원 이하의 본인 돈을 예금해도 세금에 변화가 없다...,질문: 자녀가 어른이 아닌 경우 증여세 감면 범위는 얼마인가?\n\ndocs:\nd...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","자녀가 어른이 아닌 경우, 즉 미성년 자녀의 증여세 감면 범위는 10년 합산 기준으...",[doc1]
294,이른둥이 전용 기저귀를 국내에 처음 선보인 기업은?,[저출산에 따른 인구 정체로 어려움을 겪고 있는 생활용품·어린이용품 업체들이 ‘베이...,질문: 이른둥이 전용 기저귀를 국내에 처음 선보인 기업은?\n\ndocs:\ndoc...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",이른둥이 전용 기저귀를 국내에 처음 선보인 기업은 유한킴벌리입니다. 유한킴벌리의 기...,[doc1]
295,2010년에 카리루를 차단한 도서관은?,[서울 북촌에 자리잡은 서울교육청 정독도서관은 옛 학교 건물을 그대로 물려받았다. ...,질문: 2010년에 카리루를 차단한 도서관은?\n\ndocs:\ndoc1: 서울 북...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",2010년에 카리루를 차단한 도서관은 오카자키 시립 중앙도서관입니다. 카리루는 이 ...,[doc3]


In [ ]:
# 저장할 컬럼만 추출
save_cols = ['question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']

# 파일로 저장 (utf-8, 인덱스 없이)
df_with_citation.to_csv(
    'klue_mrc_prompt_docs_1_4_answer_citations.csv',
    columns=save_cols,
    index=False,
    encoding='utf-8'
)

In [ ]:
from google.colab import files
files.download('klue_mrc_prompt_docs_1_4_answer_citations.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>